In [ ]:
# --- Runtime setup
!pip -q install torch==2.4.1 torchmetrics==1.4.0.post0

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os, numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# --- Paths & globals
PROJ = "/content/drive/MyDrive/dissertation"
DATA_DIR = f"{PROJ}/data"
OUT_DIR  = f"{PROJ}/outputs/entity_embeddings"
NOTES    = f"{PROJ}/outputs/embedding_run_notes.txt"
os.makedirs(OUT_DIR, exist_ok=True)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EMB_DIM = 64      # keep consistent with 03a
BATCH   = 512
EPOCHS  = 8
LR      = 1e-3
WEIGHT_DECAY = 1e-5
SEED    = 42
np.random.seed(SEED); torch.manual_seed(SEED)


Mounted at /content/drive


In [ ]:
def split_tvts(X, y, test_size=0.4, seed=SEED):
    X_tr, X_tmp, y_tr, y_tmp = train_test_split(X, y, test_size=test_size, stratify=y, random_state=seed)
    X_va, X_te, y_va, y_te = train_test_split(X_tmp, y_tmp, test_size=0.5, stratify=y_tmp, random_state=seed)
    return (X_tr, y_tr), (X_va, y_va), (X_te, y_te)

class TabDataset(Dataset):
    def __init__(self, num, cats, y):
        self.num = num.astype(np.float32)
        self.cats = [c.astype(np.int64) for c in cats]
        self.y = y.astype(np.float32)
    def __len__(self): return len(self.y)
    def __getitem__(self, i):
        return self.num[i], [c[i] for c in self.cats], self.y[i]

class EntityNet(nn.Module):
    def __init__(self, cardinalities, num_in, emb_dim=64):
        super().__init__()
        self.embs = nn.ModuleList([nn.Embedding(card, emb_dim) for card in cardinalities])
        hidden_in = emb_dim * len(cardinalities) + (num_in if num_in>0 else 0)
        hidden = max(128, hidden_in // 2)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_in, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    def forward(self, x_num, x_cat_list):
        embs = [emb(x) for emb, x in zip(self.embs, x_cat_list)]
        if x_num is None or x_num.shape[1] == 0:
            x = torch.cat(embs, dim=1)
        else:
            x = torch.cat(embs + [x_num], dim=1)
        return self.mlp(x).squeeze(1)

def train_entity_embeddings(dataset_name, csv_path, cats, target_col,
                            leakage_blacklist=None, force_keep_numeric=None):
    # ---------------- Load & clean
    df = pd.read_csv(csv_path, low_memory=False)
    df = df.loc[:, ~df.columns.duplicated()]
    for c in cats:
        df[c] = df[c].astype(str).str.strip().replace({"": np.nan}).fillna("Unknown")

    # ---------------- Numeric selection (exclude target & leakage)
    leak = set(leakage_blacklist or [])
    num_cols = df.select_dtypes(include=["int64","float64"]).columns.tolist()
    if target_col in num_cols: num_cols.remove(target_col)
    num_cols = [c for c in num_cols if c not in leak and c != target_col]
    if force_keep_numeric:
        num_cols = [c for c in num_cols if c in set(force_keep_numeric)]
    if len(num_cols) > 0:
        for c in num_cols:
            df[c] = pd.to_numeric(df[c], errors="coerce")
        df[num_cols] = df[num_cols].replace([np.inf, -np.inf], np.nan)
        df[num_cols] = df[num_cols].fillna(df[num_cols].median())

    # ---------------- Map cats -> ids
    id_maps = {}
    for c in cats:
        vals = df[c].astype("category")
        id_maps[c] = dict(enumerate(vals.cat.categories))
        df[c] = vals.cat.codes  # 0..n-1

    # ---------------- Arrays & splits
    y = df[target_col].astype(int).values
    X_cats = [df[c].values for c in cats]
    X_num  = df[num_cols].values if len(num_cols)>0 else np.empty((len(df), 0), dtype=np.float32)

    (Xn_tr, y_tr), (Xn_va, y_va), (Xn_te, y_te) = split_tvts(X_num, y)

    def split_cat(arr):
        return (
            arr[0:len(Xn_tr)],
            arr[len(Xn_tr):len(Xn_tr)+len(Xn_va)],
            arr[len(Xn_tr)+len(Xn_va):]
        )
    Xc_tr, Xc_va, Xc_te = zip(*[split_cat(arr) for arr in X_cats])

    ds_tr = TabDataset(Xn_tr, list(Xc_tr), y_tr)
    ds_va = TabDataset(Xn_va, list(Xc_va), y_va)
    dl_tr = DataLoader(ds_tr, batch_size=BATCH, shuffle=True)
    dl_va = DataLoader(ds_va, batch_size=BATCH, shuffle=False)

    # ---------------- Model, optimizer, BCE with pos_weight
    cards = [int(df[c].max()+1) for c in cats]
    model = EntityNet(cards, Xn_tr.shape[1], emb_dim=EMB_DIM).to(DEVICE)
    opt = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

    pos = y_tr.mean(); neg = 1 - pos
    pos_weight = torch.tensor([neg/pos], device=DEVICE)  # >1 boosts positives
    loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    # ---------------- Epoch loop (track best val AUC)
    def run_epoch(dl, train=True, return_probs=False):
        model.train() if train else model.eval()
        total_loss, yh, yt = 0.0, [], []
        for xb_num, xb_cats, yb in dl:
            xb_num = xb_num.to(DEVICE)
            xb_cats = [t.to(DEVICE) for t in xb_cats]
            yb = yb.to(DEVICE)
            with torch.set_grad_enabled(train):
                logits = model(xb_num, xb_cats)
                loss = loss_fn(logits, yb)
                if train:
                    opt.zero_grad(); loss.backward(); opt.step()
            total_loss += loss.item() * len(yb)
            yh.append(torch.sigmoid(logits).detach().cpu().numpy())
            yt.append(yb.detach().cpu().numpy())
        p = np.concatenate(yh); ytrue = np.concatenate(yt)
        pred05 = (p > 0.5).astype(int)
        out = dict(
            LOSS=total_loss/len(ytrue),
            AUC=roc_auc_score(ytrue, p),
            F1=f1_score(ytrue, pred05, zero_division=0),
            ACC=accuracy_score(ytrue, pred05),
            PREC=precision_score(ytrue, pred05, zero_division=0),
            REC=recall_score(ytrue, pred05),
        )
        if return_probs:
            out["PROBS"] = p; out["YTRUE"] = ytrue
        return out

    best, best_state = None, None
    for epoch in range(1, EPOCHS+1):
        tr = run_epoch(dl_tr, train=True)
        va = run_epoch(dl_va, train=False, return_probs=True)
        if (best is None) or (va["AUC"] > best["AUC"]):
            best, best_state = va.copy(), model.state_dict()
        print(f"[{dataset_name}] ep {epoch}/{EPOCHS}  trAUC={tr['AUC']:.3f} vaAUC={va['AUC']:.3f} vaF1@0.5={va['F1']:.3f}")

    # ---------------- Threshold tuning on val for reporting
    model.load_state_dict(best_state)
    va = best
    grid = np.linspace(0.1, 0.9, 33)
    scores = [(t, f1_score(va["YTRUE"], (va["PROBS"]>t).astype(int), zero_division=0)) for t in grid]
    thr, f1best = max(scores, key=lambda x: x[1])
    print(f"[{dataset_name}] tuned threshold={thr:.2f} best F1={f1best:.3f} (vs F1@0.5={va['F1']:.3f})")

    # ---------------- Export embedding matrices (for the best state)
    for emb_layer, c, card in zip(model.embs, cats, cards):
        E = emb_layer.weight.detach().cpu().numpy()
        inv = {i: lbl for i, lbl in id_maps[c].items()}
        idx = [f"{c}:{inv.get(i,'<unk>')}" for i in range(card)]
        out = pd.DataFrame(E, index=idx)
        out.index.name = "token"
        path = os.path.join(OUT_DIR, f"{dataset_name}_{c}_entity_{EMB_DIM}d.csv")
        out.to_csv(path)
        print("💾 saved", path, out.shape)

    # ---------------- Log note
    with open(NOTES, "a") as f:
        f.write(f"{dataset_name} entity embeddings (dim={EMB_DIM}): cats={cats}, "
                f"valAUC_best={float(va['AUC']):.3f}, valF1@0.5={va['F1']:.3f}, tunedThr={thr:.2f}, tunedF1={f1best:.3f}\n")

    # minimal return
    return dict(AUC=float(va["AUC"]), F1_05=float(va["F1"]), F1_best=float(f1best), thr=float(thr))


In [ ]:
# --- Adult
adult_res = train_entity_embeddings(
    dataset_name="adult",
    csv_path=f"{DATA_DIR}/Adult_clean.csv",
    cats=["occupation","workclass","native_country"],
    target_col="income"
)
adult_res


[adult] ep 1/8  trAUC=0.700 vaAUC=0.729 vaF1@0.5=0.483
[adult] ep 2/8  trAUC=0.758 vaAUC=0.779 vaF1@0.5=0.495
[adult] ep 3/8  trAUC=0.791 vaAUC=0.827 vaF1@0.5=0.563
[adult] ep 4/8  trAUC=0.822 vaAUC=0.830 vaF1@0.5=0.585
[adult] ep 5/8  trAUC=0.834 vaAUC=0.827 vaF1@0.5=0.567
[adult] ep 6/8  trAUC=0.839 vaAUC=0.839 vaF1@0.5=0.581
[adult] ep 7/8  trAUC=0.836 vaAUC=0.822 vaF1@0.5=0.549
[adult] ep 8/8  trAUC=0.837 vaAUC=0.834 vaF1@0.5=0.589
[adult] tuned threshold=0.60 best F1=0.594 (vs F1@0.5=0.581)
💾 saved /content/drive/MyDrive/dissertation/outputs/entity_embeddings/adult_occupation_entity_64d.csv (15, 64)
💾 saved /content/drive/MyDrive/dissertation/outputs/entity_embeddings/adult_workclass_entity_64d.csv (9, 64)
💾 saved /content/drive/MyDrive/dissertation/outputs/entity_embeddings/adult_native_country_entity_64d.csv (42, 64)


{'AUC': 0.8388156635109475,
 'F1_05': 0.5805417534861356,
 'F1_best': 0.5939675174013921,
 'thr': 0.6}

In [ ]:
# --- PetFinder
pet_res = train_entity_embeddings(
    dataset_name="petfinder",
    csv_path=f"{DATA_DIR}/Petfinder_clean.csv",
    cats=["Breed1","Color1","MaturitySize"],
    target_col="AdoptionSpeed_bin"
)
pet_res


[petfinder] ep 1/8  trAUC=0.537 vaAUC=0.563 vaF1@0.5=0.373
[petfinder] ep 2/8  trAUC=0.589 vaAUC=0.565 vaF1@0.5=0.389
[petfinder] ep 3/8  trAUC=0.585 vaAUC=0.554 vaF1@0.5=0.361
[petfinder] ep 4/8  trAUC=0.600 vaAUC=0.566 vaF1@0.5=0.384
[petfinder] ep 5/8  trAUC=0.614 vaAUC=0.574 vaF1@0.5=0.386
[petfinder] ep 6/8  trAUC=0.613 vaAUC=0.577 vaF1@0.5=0.400
[petfinder] ep 7/8  trAUC=0.617 vaAUC=0.577 vaF1@0.5=0.385
[petfinder] ep 8/8  trAUC=0.625 vaAUC=0.574 vaF1@0.5=0.369
[petfinder] tuned threshold=0.50 best F1=0.400 (vs F1@0.5=0.400)
💾 saved /content/drive/MyDrive/dissertation/outputs/entity_embeddings/petfinder_Breed1_entity_64d.csv (166, 64)
💾 saved /content/drive/MyDrive/dissertation/outputs/entity_embeddings/petfinder_Color1_entity_64d.csv (7, 64)
💾 saved /content/drive/MyDrive/dissertation/outputs/entity_embeddings/petfinder_MaturitySize_entity_64d.csv (3, 64)


{'AUC': 0.5774820388950823,
 'F1_05': 0.399801093983093,
 'F1_best': 0.399801093983093,
 'thr': 0.5}

In [ ]:
# --- Breast (with leakage guard)
LEAK_BREAST = {
    "DX_LASTCONTACT_DEATH_MONTHS", "OS5yr", "OS5yr_bin",
    "PUF_30_DAY_MORT_CD", "PUF_90_DAY_MORT_CD", "PUF_VITAL_STATUS",
    "READM_HOSP_30_DAYS", "RX_SUMM_SURGRAD_SEQ", "RX_SUMM_SYSTEMIC_SUR_SEQ",
    "died", "five_yr_surv", "followup_months"
}
breast_res = train_entity_embeddings(
    dataset_name="breast",
    csv_path=f"{DATA_DIR}/Breast_clean.csv",
    cats=["TNM_PATH_T","TNM_PATH_N","hospid"],
    target_col="OS5yr_bin",
    leakage_blacklist=LEAK_BREAST
)
breast_res


[breast] ep 1/8  trAUC=0.567 vaAUC=0.614 vaF1@0.5=0.480
[breast] ep 2/8  trAUC=0.618 vaAUC=0.630 vaF1@0.5=0.150
[breast] ep 3/8  trAUC=0.618 vaAUC=0.630 vaF1@0.5=0.247
[breast] ep 4/8  trAUC=0.620 vaAUC=0.631 vaF1@0.5=0.247
[breast] ep 5/8  trAUC=0.635 vaAUC=0.614 vaF1@0.5=0.476
[breast] ep 6/8  trAUC=0.636 vaAUC=0.631 vaF1@0.5=0.473
[breast] ep 7/8  trAUC=0.643 vaAUC=0.639 vaF1@0.5=0.496
[breast] ep 8/8  trAUC=0.644 vaAUC=0.628 vaF1@0.5=0.488
[breast] tuned threshold=0.58 best F1=0.501 (vs F1@0.5=0.496)
💾 saved /content/drive/MyDrive/dissertation/outputs/entity_embeddings/breast_TNM_PATH_T_entity_64d.csv (25, 64)
💾 saved /content/drive/MyDrive/dissertation/outputs/entity_embeddings/breast_TNM_PATH_N_entity_64d.csv (31, 64)
💾 saved /content/drive/MyDrive/dissertation/outputs/entity_embeddings/breast_hospid_entity_64d.csv (1318, 64)


{'AUC': 0.6385349125394064,
 'F1_05': 0.4956785107483194,
 'F1_best': 0.5009068685434903,
 'thr': 0.5750000000000001}

In [ ]:
from datetime import datetime
with open(NOTES, "a") as f:
    f.write(f"[{datetime.now():%Y-%m-%d %H:%M}] 03b rerun complete (EMB_DIM={EMB_DIM})\n")
print("Logged.")


📝 Logged.


In [ ]:
from datetime import datetime
with open(NOTES, "a") as f:
    f.write(f"[{datetime.now():%Y-%m-%d %H:%M}] 03b rerun complete (EMB_DIM={EMB_DIM})\n")
print("Logged.")


📝 Logged.


In [ ]:
# Merge the 3 per-column entity CSVs into one per dataset (prefixed tokens)
import pandas as pd, os

PROJ = "/content/drive/MyDrive/dissertation"
IN_DIR = f"{PROJ}/outputs/entity_embeddings"
OUT_DIR = f"{PROJ}/outputs/entity_embeddings_merged"
os.makedirs(OUT_DIR, exist_ok=True)

def merge_entity_csvs(dataset, cols, emb_dim=64):
    dfs = []
    for c in cols:
        path = os.path.join(IN_DIR, f"{dataset}_{c}_entity_{emb_dim}d.csv")
        df = pd.read_csv(path)
        # ensure 'token' is first col; keep as index-like
        if df.columns[0] != "token":
            df.rename(columns={df.columns[0]:"token"}, inplace=True)
        dfs.append(df)
    merged = pd.concat(dfs, axis=0, ignore_index=True)
    out = os.path.join(OUT_DIR, f"{dataset}_entity_{emb_dim}d_merged.csv")
    merged.to_csv(out, index=False)
    print("merged:", out, merged.shape)

merge_entity_csvs("adult",     ["occupation","workclass","native_country"], emb_dim=64)
merge_entity_csvs("petfinder", ["Breed1","Color1","MaturitySize"],          emb_dim=64)
merge_entity_csvs("breast",    ["TNM_PATH_T","TNM_PATH_N","hospid"],        emb_dim=64)


✅ merged: /content/drive/MyDrive/dissertation/outputs/entity_embeddings_merged/adult_entity_64d_merged.csv (66, 65)
✅ merged: /content/drive/MyDrive/dissertation/outputs/entity_embeddings_merged/petfinder_entity_64d_merged.csv (176, 65)
✅ merged: /content/drive/MyDrive/dissertation/outputs/entity_embeddings_merged/breast_entity_64d_merged.csv (1374, 65)


In [ ]:
!pip -q install lightgbm==4.5.0 category_encoders==2.6.3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
import os, numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression

import lightgbm as lgb

PROJ = "/content/drive/MyDrive/dissertation"
DATA_DIR = f"{PROJ}/data"
EMB_MERGED_DIR = f"{PROJ}/outputs/entity_embeddings_merged"
OUT_DIR = f"{PROJ}/outputs/bridge_eval"
os.makedirs(OUT_DIR, exist_ok=True)

def load_merged_entity(path_csv):
    df = pd.read_csv(path_csv)
    if df.columns[0] != "token":  # ensure token column
        df = df.rename(columns={df.columns[0]:"token"})
    df = df.set_index("token")
    return df

def cats_for_dataset(name):
    if name == "adult":
        return ["occupation","workclass","native_country"], "income"
    if name == "petfinder":
        return ["Breed1","Color1","MaturitySize"], "AdoptionSpeed_bin"
    if name == "breast":
        return ["TNM_PATH_T","TNM_PATH_N","hospid"], "OS5yr_bin"
    raise ValueError(name)

def leakage_blacklist(name):
    if name != "breast":
        return set()
    return {
        "DX_LASTCONTACT_DEATH_MONTHS","OS5yr","OS5yr_bin",
        "PUF_30_DAY_MORT_CD","PUF_90_DAY_MORT_CD","PUF_VITAL_STATUS",
        "READM_HOSP_30_DAYS","RX_SUMM_SURGRAD_SEQ","RX_SUMM_SYSTEMIC_SUR_SEQ",
        "died","five_yr_surv","followup_months"
    }

def make_row_vectors(df, cats, emb_table):  # concat 3×64 = 192
    toks = []
    for c in cats:
        tok = (c + ":" + df[c].astype(str).str.strip().replace({"":"Unknown"}).fillna("Unknown"))
        toks.append(tok.values)
    emb_arrays = []
    for t in toks:
        # look up; unseen → zeros
        V = np.zeros((len(t), emb_table.shape[1]), dtype=np.float32)
        idx = emb_table.index
        take = idx.get_indexer(t)
        hit = take >= 0
        V[hit] = emb_table.iloc[take[hit]].values
        emb_arrays.append(V)
    return np.concatenate(emb_arrays, axis=1)

def prep_dataset(name):
    cats, target = cats_for_dataset(name)
    # load clean CSV
    clean_path = f"{DATA_DIR}/{name.capitalize()}_clean.csv" if name!="petfinder" else f"{DATA_DIR}/Petfinder_clean.csv"
    df = pd.read_csv(clean_path, low_memory=False)
    # numerics
    leak = leakage_blacklist(name)
    num_cols = df.select_dtypes(include=["int64","float64"]).columns.tolist()
    if target in num_cols: num_cols.remove(target)
    num_cols = [c for c in num_cols if c not in leak]
    # impute numeric
    for c in num_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df[num_cols] = df[num_cols].replace([np.inf,-np.inf], np.nan)
    df[num_cols] = df[num_cols].fillna(df[num_cols].median())

    # entity vectors
    emb = load_merged_entity(os.path.join(EMB_MERGED_DIR, f"{name}_entity_64d_merged.csv"))
    # ensure cats exist
    for c in cats:
        df[c] = df[c].astype(str).str.strip().replace({"":np.nan}).fillna("Unknown")

    X_emb = make_row_vectors(df, cats, emb)
    X_num = df[num_cols].values if len(num_cols)>0 else np.empty((len(df),0),dtype=np.float32)
    X = np.concatenate([X_num, X_emb], axis=1)

    y = df[target].astype(int).values
    X_tr, X_tmp, y_tr, y_tmp = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)
    X_va, X_te, y_va, y_te = train_test_split(X_tmp, y_tmp, test_size=0.5, stratify=y_tmp, random_state=42)
    return (X_tr,y_tr),(X_va,y_va),(X_te,y_te), num_cols

def score_probs(p, y, label):
    pred05 = (p>0.5).astype(int)
    res = dict(
        Model=label,
        AUC = roc_auc_score(y, p),
        F1  = f1_score(y, pred05, zero_division=0),
        ACC = accuracy_score(y, pred05),
        PREC= precision_score(y, pred05, zero_division=0),
        REC = recall_score(y, pred05)
    )
    # tuned threshold for F1
    grid = np.linspace(0.1,0.9,33)
    thr, best = 0.5, -1
    for t in grid:
        f = f1_score(y, (p>t).astype(int), zero_division=0)
        if f>best: thr, best = t, f
    res["F1_best"] = best
    res["thr_best"] = thr
    return res

def run_bridge(name):
    (X_tr,y_tr),(X_va,y_va),(X_te,y_te), num_cols = prep_dataset(name)

    # Logistic Regression (balanced)
    lr = LogisticRegression(max_iter=500, class_weight="balanced", n_jobs=-1)
    lr.fit(X_tr, y_tr)
    lr_res = score_probs(lr.predict_proba(X_va)[:,1], y_va, "LR+Entity(192)+Num")

    # LightGBM (good with mixed dense features)
    params = dict(
        objective="binary",
        boosting_type="gbdt",
        learning_rate=0.05,
        num_leaves=31,
        feature_fraction=0.9,
        bagging_fraction=0.8,
        bagging_freq=1,
        n_estimators=600,
        reg_lambda=1.0,
        verbose=-1
    )
    # handle class imbalance
    pos_weight = (1 - y_tr.mean())/y_tr.mean()
    params["scale_pos_weight"] = pos_weight

    gbm = lgb.LGBMClassifier(**params)
    gbm.fit(X_tr, y_tr, eval_set=[(X_va,y_va)], eval_metric="auc", callbacks=[lgb.early_stopping(50, verbose=False)])
    gbm_res = score_probs(gbm.predict_proba(X_va)[:,1], y_va, "LGBM+Entity(192)+Num")

    df_res = pd.DataFrame([lr_res, gbm_res])
    out = os.path.join(OUT_DIR, f"{name}_bridge_entity.csv")
    df_res.to_csv(out, index=False)
    print(f"{name} results saved → {out}")
    display(df_res)
    return df_res

all_res = []
for ds in ["adult","petfinder","breast"]:
    print(f"\n=== {ds.upper()} ===")
    r = run_bridge(ds)
    r.insert(0, "Dataset", ds)
    all_res.append(r)

summary = pd.concat(all_res, axis=0, ignore_index=True)
summary_out = os.path.join(OUT_DIR, "bridge_entity_summary.csv")
summary.to_csv(summary_out, index=False)
print("\nSummary saved →", summary_out)
display(summary)



=== ADULT ===


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


✅ adult results saved → /content/drive/MyDrive/dissertation/outputs/bridge_eval/adult_bridge_entity.csv


,Model,AUC,F1,ACC,PREC,REC,F1_best,thr_best
0,LR+Entity(192)+Num,0.844731,0.603304,0.761568,0.501132,0.757809,0.613522,0.55
1,LGBM+Entity(192)+Num,0.886784,0.640827,0.786548,0.536332,0.795892,0.665658,0.65



=== PETFINDER ===


...

/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


✅ breast results saved → /content/drive/MyDrive/dissertation/outputs/bridge_eval/breast_bridge_entity.csv


,Model,AUC,F1,ACC,PREC,REC,F1_best,thr_best
0,LR+Entity(192)+Num,0.665101,0.502232,0.598980,0.400563,0.673066,0.508441,0.425
1,LGBM+Entity(192)+Num,0.942126,0.812207,0.866187,0.702395,0.962716,0.816471,0.625



✅ Summary saved → /content/drive/MyDrive/dissertation/outputs/bridge_eval/bridge_entity_summary.csv


,Dataset,Model,AUC,F1,ACC,PREC,REC,F1_best,thr_best
0,adult,LR+Entity(192)+Num,0.844731,0.603304,0.761568,0.501132,0.757809,0.613522,0.550
1,adult,LGBM+Entity(192)+Num,0.886784,0.640827,0.786548,0.536332,0.795892,0.665658,0.650
2,petfinder,LR+Entity(192)+Num,0.645933,0.407303,0.634157,0.332569,0.525362,0.434590,0.450
3,petfinder,LGBM+Entity(192)+Num,0.675982,0.000000,0.760728,0.000000,0.000000,0.448157,0.300
4,breast,LR+Entity(192)+Num,0.665101,0.502232,0.598980,0.400563,0.673066,0.508441,0.425
5,breast,LGBM+Entity(192)+Num,0.942126,0.812207,0.866187,0.702395,0.962716,0.816471,0.625


In [ ]:
# === Save val predictions at 0.5 and tuned threshold for LR & LGBM (Entity bridge) ===
import os, numpy as np, pandas as pd
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

PROJ = "/content/drive/MyDrive/dissertation"
DATA_DIR = f"{PROJ}/data"
EMB_MERGED_DIR = f"{PROJ}/outputs/entity_embeddings_merged"
PRED_DIR = f"{PROJ}/outputs/bridge_eval/preds"
os.makedirs(PRED_DIR, exist_ok=True)

def load_merged_entity(path_csv):
    df = pd.read_csv(path_csv)
    if df.columns[0] != "token":
        df = df.rename(columns={df.columns[0]:"token"})
    return df.set_index("token")

def cats_for_dataset(name):
    if name == "adult":      return ["occupation","workclass","native_country"], "income"
    if name == "petfinder":  return ["Breed1","Color1","MaturitySize"], "AdoptionSpeed_bin"
    if name == "breast":     return ["TNM_PATH_T","TNM_PATH_N","hospid"], "OS5yr_bin"
    raise ValueError(name)

def leakage_blacklist(name):
    return set() if name!="breast" else {
        "DX_LASTCONTACT_DEATH_MONTHS","OS5yr","OS5yr_bin",
        "PUF_30_DAY_MORT_CD","PUF_90_DAY_MORT_CD","PUF_VITAL_STATUS",
        "READM_HOSP_30_DAYS","RX_SUMM_SURGRAD_SEQ","RX_SUMM_SYSTEMIC_SUR_SEQ",
        "died","five_yr_surv","followup_months"
    }

def make_row_vectors(df, cats, emb_table):
    toks = []
    for c in cats:
        tok = (c + ":" + df[c].astype(str).str.strip().replace({"":"Unknown"}).fillna("Unknown"))
        toks.append(tok.values)
    emb_arrays = []
    for t in toks:
        V = np.zeros((len(t), emb_table.shape[1]), dtype=np.float32)
        idx = emb_table.index
        take = idx.get_indexer(t)
        hit = take >= 0
        V[hit] = emb_table.iloc[take[hit]].values
        emb_arrays.append(V)
    return np.concatenate(emb_arrays, axis=1)

def prep_dataset(name):
    cats, target = cats_for_dataset(name)
    clean_path = f"{DATA_DIR}/{name.capitalize()}_clean.csv" if name!="petfinder" else f"{DATA_DIR}/Petfinder_clean.csv"
    df = pd.read_csv(clean_path, low_memory=False)

    leak = leakage_blacklist(name)
    num_cols = df.select_dtypes(include=["int64","float64"]).columns.tolist()
    if target in num_cols: num_cols.remove(target)
    num_cols = [c for c in num_cols if c not in leak]
    # impute numeric
    for c in num_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df[num_cols] = df[num_cols].replace([np.inf,-np.inf], np.nan)
    df[num_cols] = df[num_cols].fillna(df[num_cols].median())

    emb = load_merged_entity(os.path.join(EMB_MERGED_DIR, f"{name}_entity_64d_merged.csv"))
    for c in cats:
        df[c] = df[c].astype(str).str.strip().replace({"":np.nan}).fillna("Unknown")

    X_emb = make_row_vectors(df, cats, emb)
    X_num = df[num_cols].values if len(num_cols)>0 else np.empty((len(df),0),dtype=np.float32)
    X = np.concatenate([X_num, X_emb], axis=1)
    y = df[cats_for_dataset(name)[1]].astype(int).values

    from sklearn.model_selection import train_test_split
    X_tr, X_tmp, y_tr, y_tmp = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)
    X_va, X_te, y_va, y_te = train_test_split(X_tmp, y_tmp, test_size=0.5, stratify=y_tmp, random_state=42)
    return (X_tr,y_tr),(X_va,y_va)

def tuned_threshold(probs, y):
    grid = np.linspace(0.1,0.9,33)
    best_t, best_f1 = 0.5, -1
    for t in grid:
        f = f1_score(y, (probs>t).astype(int), zero_division=0)
        if f>best_f1: best_t, best_f1 = t, f
    return best_t, best_f1

def save_preds_for(name):
    (X_tr,y_tr),(X_va,y_va) = prep_dataset(name)

    # LR
    lr = LogisticRegression(max_iter=500, class_weight="balanced", n_jobs=-1).fit(X_tr, y_tr)
    p_lr = lr.predict_proba(X_va)[:,1]
    thr_lr, f1_lr = tuned_threshold(p_lr, y_va)
    cm_lr = confusion_matrix(y_va, (p_lr>thr_lr).astype(int))
    print(f"[{name}] LR tuned thr={thr_lr:.2f} F1={f1_lr:.3f} CM:\n{cm_lr}")

    # LGBM
    import lightgbm as lgb
    w = (1 - y_tr.mean())/y_tr.mean()
    gbm = lgb.LGBMClassifier(objective="binary", learning_rate=0.05, num_leaves=31,
                             feature_fraction=0.9, bagging_fraction=0.8, bagging_freq=1,
                             n_estimators=600, reg_lambda=1.0, scale_pos_weight=w, verbose=-1)
    gbm.fit(X_tr, y_tr, eval_set=[(X_va,y_va)], eval_metric="auc",
            callbacks=[lgb.early_stopping(50, verbose=False)])
    p_gb = gbm.predict_proba(X_va)[:,1]
    thr_gb, f1_gb = tuned_threshold(p_gb, y_va)
    cm_gb = confusion_matrix(y_va, (p_gb>thr_gb).astype(int))
    print(f"[{name}] LGBM tuned thr={thr_gb:.2f} F1={f1_gb:.3f} CM:\n{cm_gb}")

    # Save tidy preds
    df = pd.DataFrame({
        "y_true": y_va,
        "prob_LR": p_lr, "pred_LR_0p5": (p_lr>0.5).astype(int), "pred_LR_best": (p_lr>thr_lr).astype(int),
        "prob_LGBM": p_gb, "pred_LGBM_0p5": (p_gb>0.5).astype(int), "pred_LGBM_best": (p_gb>thr_gb).astype(int),
    })
    df.attrs["thr_LR_best"] = thr_lr
    df.attrs["thr_LGBM_best"] = thr_gb
    out = os.path.join(PRED_DIR, f"{name}_entity_preds_val.csv")
    df.to_csv(out, index=False)
    print("saved preds →", out)

for ds in ["adult","petfinder","breast"]:
    print("\n===", ds.upper(), "===")
    save_preds_for(ds)



=== ADULT ===
[adult] LR tuned thr=0.55 F1=0.614 CM:
[[6046 1385]
 [ 690 1647]]


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[adult] LGBM tuned thr=0.65 F1=0.666 CM:
[[6681  750]
 [ 797 1540]]
✅ saved preds → /content/drive/MyDrive/dissertation/outputs/bridge_eval/preds/adult_entity_preds_val.csv

=== PETFINDER ===
[petfinder] LR tuned thr=0.45 F1=0.435 CM:
[[895 860]
 [160 392]]


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[petfinder] LGBM tuned thr=0.30 F1=0.448 CM:
[[960 795]
 [163 389]]
✅ saved preds → /content/drive/MyDrive/dissertation/outputs/bridge_eval/preds/petfinder_entity_preds_val.csv

=== BREAST ===
[breast] LR tuned thr=0.43 F1=0.508 CM:
[[3033 5330]
 [ 552 3042]]


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[breast] LGBM tuned thr=0.62 F1=0.816 CM:
[[7114 1249]
 [ 253 3341]]
✅ saved preds → /content/drive/MyDrive/dissertation/outputs/bridge_eval/preds/breast_entity_preds_val.csv


In [ ]:
# === Bridge eval for 03a: W2V / FastText / Node2Vec ===
!pip -q install lightgbm==4.5.0

import os, numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

PROJ = "/content/drive/MyDrive/dissertation"
DATA_DIR = f"{PROJ}/data"
EMB_DIR = f"{PROJ}/outputs/embeddings"
OUT_DIR = f"{PROJ}/outputs/bridge_eval_03a"
os.makedirs(OUT_DIR, exist_ok=True)

METHOD_FILES = {
    "w2v":      "{ds}_w2v_64d_prefixed.csv",
    "fasttext": "{ds}_fasttext_64d_prefixed.csv",
    "node2vec": "{ds}_node2vec_64d_prefixed.csv",
}

def cats_for_dataset(name):
    if name == "adult":
        return ["occupation","workclass","native_country"], "income"
    if name == "petfinder":
        return ["Breed1","Color1","MaturitySize"], "AdoptionSpeed_bin"
    if name == "breast":
        return ["TNM_PATH_T","TNM_PATH_N","hospid"], "OS5yr_bin"
    raise ValueError(name)

def leakage_blacklist(name):
    return set() if name!="breast" else {
        "DX_LASTCONTACT_DEATH_MONTHS","OS5yr","OS5yr_bin",
        "PUF_30_DAY_MORT_CD","PUF_90_DAY_MORT_CD","PUF_VITAL_STATUS",
        "READM_HOSP_30_DAYS","RX_SUMM_SURGRAD_SEQ","RX_SUMM_SYSTEMIC_SUR_SEQ",
        "died","five_yr_surv","followup_months"
    }

def load_token_table(path_csv):
    df = pd.read_csv(path_csv)
    if df.columns[0] != "token":
        df = df.rename(columns={df.columns[0]:"token"})
    return df.set_index("token")

def make_row_vectors(df, cats, tok_table):
    toks = []
    for c in cats:
        tok = (c + ":" + df[c].astype(str).str.strip().replace({"":"Unknown"}).fillna("Unknown"))
        toks.append(tok.values)
    emb_arrays = []
    dim = tok_table.shape[1]
    idx = tok_table.index
    for t in toks:
        V = np.zeros((len(t), dim), dtype=np.float32)
        take = idx.get_indexer(t)
        hit = take >= 0
        V[hit] = tok_table.iloc[take[hit]].values
        emb_arrays.append(V)
    return np.concatenate(emb_arrays, axis=1)  # 3*64=192

def prep_dataset(name, token_table):
    cats, target = cats_for_dataset(name)
    clean_path = f"{DATA_DIR}/{name.capitalize()}_clean.csv" if name!="petfinder" else f"{DATA_DIR}/Petfinder_clean.csv"
    df = pd.read_csv(clean_path, low_memory=False)

    leak = leakage_blacklist(name)
    num_cols = df.select_dtypes(include=["int64","float64"]).columns.tolist()
    if target in num_cols: num_cols.remove(target)
    num_cols = [c for c in num_cols if c not in leak]
    for c in num_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df[num_cols] = df[num_cols].replace([np.inf,-np.inf], np.nan)
    df[num_cols] = df[num_cols].fillna(df[num_cols].median())

    for c in cats:
        df[c] = df[c].astype(str).str.strip().replace({"":np.nan}).fillna("Unknown")

    X_emb = make_row_vectors(df, cats, token_table)
    X_num = df[num_cols].values if len(num_cols)>0 else np.empty((len(df),0),dtype=np.float32)
    X = np.concatenate([X_num, X_emb], axis=1)
    y = df[target].astype(int).values

    X_tr, X_tmp, y_tr, y_tmp = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)
    X_va, X_te, y_va, y_te = train_test_split(X_tmp, y_tmp, test_size=0.5, stratify=y_tmp, random_state=42)
    return (X_tr,y_tr),(X_va,y_va)

def score_probs(p, y, label):
    pred05 = (p>0.5).astype(int)
    res = dict(
        Model=label,
        AUC = roc_auc_score(y, p),
        F1  = f1_score(y, pred05, zero_division=0),
        ACC = accuracy_score(y, pred05),
        PREC= precision_score(y, pred05, zero_division=0),
        REC = recall_score(y, pred05)
    )
    grid = np.linspace(0.1,0.9,33)
    thr, best = 0.5, -1
    for t in grid:
        f = f1_score(y, (p>t).astype(int), zero_division=0)
        if f>best: thr, best = t, f
    res["F1_best"] = best
    res["thr_best"] = thr
    return res

def run_03a_bridge(ds, method):
    file = METHOD_FILES[method].format(ds=ds)
    tok_table = load_token_table(os.path.join(EMB_DIR, file))
    (X_tr,y_tr),(X_va,y_va) = prep_dataset(ds, tok_table)

    lr = LogisticRegression(max_iter=500, class_weight="balanced", n_jobs=-1).fit(X_tr, y_tr)
    lr_res = score_probs(lr.predict_proba(X_va)[:,1], y_va, f"LR+{method.upper()}(192)+Num")

    w = (1 - y_tr.mean())/y_tr.mean()
    gbm = lgb.LGBMClassifier(objective="binary", learning_rate=0.05, num_leaves=31,
                             feature_fraction=0.9, bagging_fraction=0.8, bagging_freq=1,
                             n_estimators=600, reg_lambda=1.0, scale_pos_weight=w, verbose=-1)
    gbm.fit(X_tr, y_tr, eval_set=[(X_va,y_va)], eval_metric="auc",
            callbacks=[lgb.early_stopping(50, verbose=False)])
    gbm_res = score_probs(gbm.predict_proba(X_va)[:,1], y_va, f"LGBM+{method.upper()}(192)+Num")

    out = os.path.join(OUT_DIR, f"{ds}_{method}_bridge.csv")
    pd.DataFrame([lr_res, gbm_res]).to_csv(out, index=False)
    print(f"{ds}-{method} → {out}")

    r = pd.DataFrame([lr_res, gbm_res])
    r.insert(0, "Dataset", ds)
    r.insert(1, "Embedding", method.upper())
    return r

all_rows = []
for ds in ["adult","petfinder","breast"]:
    for method in ["w2v","fasttext","node2vec"]:
        try:
            all_rows.append(run_03a_bridge(ds, method))
        except FileNotFoundError:
            print(f"⚠️ missing embeddings for {ds}-{method}, skipping.")

summary = pd.concat(all_rows, axis=0, ignore_index=True) if all_rows else pd.DataFrame()
summary_out = os.path.join(OUT_DIR, "bridge_03a_summary.csv")
summary.to_csv(summary_out, index=False)
print("\nSummary saved →", summary_out)
display(summary.head(20))


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


✅ adult-w2v → /content/drive/MyDrive/dissertation/outputs/bridge_eval_03a/adult_w2v_bridge.csv


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


✅ adult-fasttext → /content/drive/MyDrive/dissertation/outputs/bridge_eval_03a/adult_fasttext_bridge.csv


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


✅ breast-fasttext → /content/drive/MyDrive/dissertation/outputs/bridge_eval_03a/breast_fasttext_bridge.csv


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


✅ breast-node2vec → /content/drive/MyDrive/dissertation/outputs/bridge_eval_03a/breast_node2vec_bridge.csv

✅ Summary saved → /content/drive/MyDrive/dissertation/outputs/bridge_eval_03a/bridge_03a_summary.csv


,Dataset,Embedding,Model,AUC,F1,ACC,PREC,REC,F1_best,thr_best
0,adult,W2V,LR+W2V(192)+Num,0.843061,0.600279,0.764947,0.506017,0.737698,0.609240,0.575
1,adult,W2V,LGBM+W2V(192)+Num,0.887658,0.642721,0.788186,0.538796,0.796320,0.666667,0.625
2,adult,FASTTEXT,LR+FASTTEXT(192)+Num,0.837475,0.592399,0.760647,0.499853,0.727000,0.598524,0.550
3,adult,FASTTEXT,LGBM+FASTTEXT(192)+Num,0.886819,0.639491,0.785422,0.534656,0.795464,0.668470,0.625
4,adult,NODE2VEC,LR+NODE2VEC(192)+Num,0.840871,0.594080,0.764128,0.504942,0.721438,0.605399,0.550
5,adult,NODE2VEC,LGBM+NODE2VEC(192)+Num,0.886765,0.639617,0.784500,0.533105,0.799315,0.664173,0.625
6,petfinder,W2V,LR+W2V(192)+Num,0.650056,0.412234,0.616818,0.325630,0.561594,0.439724,0.450
7,petfinder,W2V,LGBM+W2V(192)+Num,0.673420,0.000000,0.760728,0.000000,0.000000,0.457698,0.300
8,petfinder,FASTTEXT,LR+FASTTEXT(192)+Num,0.632374,0.415366,0.577807,0.310592,0.626812,0.421634,0.475
9,petfinder,FASTTEXT,LGBM+FASTTEXT(192)+Num,0.674238,0.000000,0.760728,0.000000,0.000000,0.438446,0.300
